In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import os
import tempfile
from collections import defaultdict

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import custom_object_scope
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import l1  #reg ##############

In [3]:
from qkeras import *
from qkeras import QActivation
from qkeras import QDense
from qkeras import QConv2D
from qkeras import quantized_bits, quantized_relu
from qkeras.utils import load_qmodel
from qkeras.utils import print_model_sparsity
from qkeras.utils import model_save_quantized_weights

In [34]:
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.sparsity.keras import prune
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_schedule

from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude, strip_pruning 
from tensorflow_model_optimization.sparsity.keras import ConstantSparsity
from tensorflow_model_optimization.sparsity.keras import PolynomialDecay

In [8]:
from qkeras.utils import QN...

gradual_qnoise_callback_0 = QNoiseScheduler(
      start=2, finish=4, freq_type="step", exponent=3.0)

gradual_qnoise_callback_1 = QNoiseScheduler(
      start=2, finish=10, freq_type="step", exponent=3.0)

gradual_qnoise_callback_2 = QNoiseScheduler(
      start=2, finish=10, freq_type="step", exponent=2.0)

gradual_qnoise_callback_3 = QNoiseScheduler(
      start=6, finish=10, freq_type="step", exponent=3.0)

gradual_qnoise_callback_4 = QNoiseScheduler(
      start=6, finish=20, freq_type="step", exponent=3.0)

gradual_qnoise_callback_5 = QNoiseScheduler(
      start=0, finish=20, freq_type="step", update_freq=2, exponent=3.0)

In [37]:
pruning_params = {
    'pruning_schedule': PolynomialDecay(
        initial_sparsity=0.0, 
        final_sparsity=0.5,
        begin_step=1000, end_step=10000,
        frequency=100
    )
}

#pruning_params = {
#    "pruning_schedule": ConstantSparsity(0.75, begin_step=2000, frequency=100)
#    }

In [15]:
NB_EPOCH = 15
BATCH_SIZE = 64
VERBOSE = 1
NB_CLASSES = 10
VALIDATION_SPLIT = 0.1
OPTIMIZER = Adam(learning_rate=0.001)
#OPTIMIZER = SGD(learning_rate=0.001, momentum=0.9)

W = 4  #QConv2d q_bit
I = 0

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    return lr * tf.math.exp(-0.1)

CALLBACKS = lr_schedule = LearningRateScheduler(scheduler)

In [28]:
def build_model(input_shape):
    x = x_in = Input(shape=input_shape, name="input_layer")
    a = QConv2D(
        32, (2, 2), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        kernel_regularizer=l1(1e-5),                                 #####Added L1
        name="conv2d_L1")(x)
    b = QActivation("quantized_relu(4, 0)", name="activation_1")(a)
    c = QConv2D(
        64, (3, 3), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        kernel_regularizer=l1(1e-5),
        name="conv2d_L2")(b)
    d = QActivation("quantized_relu(4, 0)", name="activation_2")(c)
    e = QConv2D(
        64, (2, 2), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        kernel_regularizer=l1(1e-5),
        name="conv2d_L3")(d)
    f = QActivation("quantized_relu(4, 0)", name="activation_3")(e)
    g = Flatten()(f)
    h = QDense(NB_CLASSES, kernel_quantizer=quantized_bits(W, I, 1),
                   bias_quantizer=quantized_bits(W, I, 1),
                   name="dense")(g)
    i = Activation("softmax", name="Softmax")(h)

    model = Model(inputs=[x_in], outputs=[i])
    return model

In [42]:
def train_and_save(model, x_train, y_train, x_test, y_test):
    model.compile(
        loss="categorical_crossentropy",
     #   loss="sparse_categorical_crossentropy",
     #   loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer= OPTIMIZER,
        metrics=["accuracy"])
    
    # Print the model summary.
    model.summary()
    
  #  callbacks=[
   #     gradual_qnoise_callback_4
    #]
    CALLBACKS = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir='pruning_logs')
    ]
    
    model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=NB_EPOCH,
        validation_split=VALIDATION_SPLIT,
        verbose=VERBOSE,
        callbacks=CALLBACKS,
        validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    print_model_sparsity(model)
    
    # Export and import the model. Check that accuracy persists.
    _, keras_file = tempfile.mkstemp(suffix=".keras")  #
    print("Saving model to:", keras_file)
    model.save("modelx.keras", save_format="keras")
    model.save_weights("modelx_w.keras")
    print("Model saved to --- .keras")
    print_qstats(model)

In [7]:
data = np.load('/home/mindspore/work/mnist.npz')
x_train, y_train = data['x_train'], data['y_train']
x_test, y_test = data['x_test'], data['y_test']

In [8]:
img_rows, img_cols = 28, 28

if K.image_data_format() == "channels_first":
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [9]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices //one-hot key 0-9 class matrice lable 0/1
y_train = to_categorical(y_train, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:
model = build_model(input_shape)
   

In [38]:
model = prune_low_magnitude(model, **pruning_params)   ##added from tfmot

In [43]:

#compile_ and_ train model and save data

train_and_save(model, x_train, y_train, x_test, y_test)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 prune_low_magnitude_Q_conv  (None, 14, 14, 32)        290       
 2d_L1 (PruneLowMagnitude)                                       
                                                                 
 prune_low_magnitude_Q_acti  (None, 14, 14, 32)        1         
 vation_1 (PruneLowMagnitud                                      
 e)                                                              
                                                                 
 prune_low_magnitude_Q_conv  (None, 6, 6, 64)          36930     
 2d_L2 (PruneLowMagnitude)                                       
                                                                 
 prune_low_magnitude_Q_acti  (None, 6, 6, 64)          1     

938/938 [==============================] - 18s 13ms/step - loss: 1.3933e-04 - accuracy: 1.0000 - val_loss: 0.0516 - val_accuracy: 0.9883
Epoch 2/15
938/938 [==============================] - 10s 11ms/step - loss: 1.0324e-04 - accuracy: 1.0000 - val_loss: 0.0524 - val_accuracy: 0.9874
Epoch 3/15
938/938 [==============================] - 10s 11ms/step - loss: 1.4355e-04 - accuracy: 1.0000 - val_loss: 0.0529 - val_accuracy: 0.9876
Epoch 4/15
938/938 [==============================] - 10s 11ms/step - loss: 4.3948e-04 - accuracy: 0.9999 - val_loss: 0.0548 - val_accuracy: 0.9867
Epoch 5/15
938/938 [==============================] - 10s 11ms/step - loss: 8.9106e-04 - accuracy: 0.9998 - val_loss: 0.0559 - val_accuracy: 0.9867
Epoch 6/15
938/938 [==============================] - 10s 11ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0584 - val_accuracy: 0.9850
Epoch 7/15
938/938 [==============================] - 10s 11ms/step - loss: 0.0020 - accuracy: 0.9996 - val_loss: 0.0545 - val_

ValueError: Please initialize `Prune` layer with a `Layer` instance. You passed: QConv2D

In [46]:
print("Reloading model")
    
#with custom_object_scope({'TFOpLambda': Lambda}):
   #     loaded_model = load_qmodel('modelx.keras')
      #  loaded_model.model_save_quantized_weights("modelx1.keras")
      #  loaded_model1 = load_qmodel("modelx1.keras")
        
with prune.prune_scope():
    loaded_model = load_qmodel('modelx.keras')

loaded_model.summary()

#loaded_model1.summary()


score = loaded_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

print("Model type - ",type(loaded_model))



Reloading model
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 prune_low_magnitude_Q_conv  (None, 14, 14, 32)        290       
 2d_L1 (PruneLowMagnitude)                                       
                                                                 
 prune_low_magnitude_Q_acti  (None, 14, 14, 32)        1         
 vation_1 (PruneLowMagnitud                                      
 e)                                                              
                                                                 
 prune_low_magnitude_Q_conv  (None, 6, 6, 64)          36930     
 2d_L2 (PruneLowMagnitude)                                       
                                                                 
 prune_low_magnitude_Q_acti  (None, 6, 6, 64)

In [47]:
print_qstats(loaded_model)
print("Model type - ",type(loaded_model))

ValueError: Please initialize `Prune` layer with a `Layer` instance. You passed: QConv2D

In [52]:

#After training, use tfmot.sparsity.keras.strip_pruning(model) to remove pruning wrappers, leaving a sparse model ready for deployment.

model3 = tfmot.sparsity.keras.strip_pruning(loaded_model)  #3x lighter

In [35]:
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.0,
    final_sparsity=0.5,
    begin_step=0,
    end_step=2000
)


In [ ]:
score = model3.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

print("Model type - ",type(model3))

In [53]:
model3.save("pruned_final_model.keras")

In [54]:
model3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 Q_conv2d_L1 (QConv2D)       (None, 14, 14, 32)        160       
                                                                 
 Q_activation_1 (QActivatio  (None, 14, 14, 32)        0         
 n)                                                              
                                                                 
 Q_conv2d_L2 (QConv2D)       (None, 6, 6, 64)          18496     
                                                                 
 Q_activation_2 (QActivatio  (None, 6, 6, 64)          0         
 n)                                                              
                                                                 
 Q_conv2d_L3 (QConv2D)       (None, 3, 3, 64)          16448 